This notebook provides examples to go along with the [textbook](https://underactuated.csail.mit.edu/lyapunov.html).  I recommend having both windows open, side-by-side!


In [ ]:
import numpy as np
from pydrake.all import (
    Jacobian,
    MathematicalProgram,
    RegionOfAttraction,
    Solve,
    SymbolicVectorSystem,
    Variable,
    Variables,
)

# Verifying a "known" ROA

First we define 
$$\dot{x} = -x + x^3, \qquad V(x) = x^2, \qquad \lambda(x) = c_0 + c_1 x + c_2 x^2, \qquad \rho = 1.$$

Then we solve
$$\text{find}_{\bf c} \qquad \text{subject to}$$
$$- \dot{V}(x) + \lambda(x) (V(x)-\rho) \text{ is SOS},$$
$$\lambda(x) \text{ is SOS}.$$

Notice that in this formulation, we've hard-coded $\rho=1$, and are just certifying the "known" ROA. If the optimization succeeds, then $V(x)\le \rho$ is inside the true region of attraction. Searching for the largest $\rho$ that can be certified in this way can be done by simply doing a line search for the largest $\rho$ for which the optimization succeeds.

In [ ]:
def sos_roa():
    prog = MathematicalProgram()
    x = prog.NewIndeterminates(1, "x")

    # Define the dynamics and Lyapunov function.
    f = -x + x**3
    V = x.dot(x)
    Vdot = Jacobian([V], x).dot(f)[0]
    rho = 1

    # Define the Lagrange multiplier.
    lambda_ = prog.NewSosPolynomial(Variables(x), 2)[0].ToExpression()

    prog.AddSosConstraint(-Vdot + lambda_ * (V - rho))

    result = Solve(prog)

    assert result.is_success(), "Optimization failed"

    print("Verified that " + str(V) + " < 1 is in the region of attraction.")


sos_roa()

# Maximizing the ROA given a Lyapunov candidates

Alternatively, we can solve for the level-set $\rho$ using the optimization
$$\max_{\bf c} \quad \rho, \qquad \text{subject to}$$
$$x^2 (V(x)-\rho) - \lambda(x)\dot{V}(x) \text{ is SOS}.$$
Note that this time, $\lambda(x)$ does not need to be SOS.

In [ ]:
def sos_roa2():
    prog = MathematicalProgram()
    x = prog.NewIndeterminates(1, "x")
    rho = prog.NewContinuousVariables(1, "rho")[0]

    # Define the dynamics and Lyapunov function.
    f = -x + x**3
    V = x.dot(x)
    Vdot = Jacobian([V], x).dot(f)[0]

    # Define the Lagrange multiplier.
    lambda_ = prog.NewFreePolynomial(Variables(x), 0).ToExpression()

    prog.AddSosConstraint((V - rho) * x.dot(x) - lambda_ * Vdot)
    prog.AddLinearCost(-rho)

    result = Solve(prog)

    assert result.is_success()

    print(
        "Verified that "
        + str(V)
        + " < "
        + str(result.GetSolution(rho))
        + " is in the region of attraction."
    )

    assert np.fabs(result.GetSolution(rho) - 1) < 1e-5


sos_roa2()

# Searching for the Lyapunov candidate

This time, we will make $V(x)$ a SOS polynomial decision variable, and search over it's coefficients.

TODO(russt): Finish this...

# Region of Attraction codes in Drake

I am hoping to make this machinery more generally useful by providing mature implementations in Drake. For many systems, you can simply call the <code>RegionOfAttraction</code> method, and it will do almost all of the work for you.

In [ ]:
def roa():
    x = Variable("x")
    sys = SymbolicVectorSystem(state=[x], dynamics=[-x + x**3])
    context = sys.CreateDefaultContext()
    V = RegionOfAttraction(system=sys, context=context)

    print("Verified that " + str(V) + " < 1 is in the region of attraction.")


roa()